In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: Averaging history and use FFN

Here, we average the full history of a path and concatenate it to the current embedding (the total number of features that are passed into the FFN is 2 * sbert_embeddings.shape[0]).

Here, we will run the hyperparameter search to implement the FFN with the same parameters as the standard FFN baseline on the sentence embeddings.

In [7]:
num_epochs = 100
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [8]:
ffn_mean_history, best_ffn_mean_history, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=None,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:31<00:00, 31.98s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


100%|██████████| 5568/5568 [00:09<00:00, 558.80it/s]


In [9]:
ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id,input_dim
0,focal,0.515729,0.515359,"[0.5019607843137254, 0.5287569573283859]",0.246686,0.686833,0.656899,"[0.7582417582417582, 0.5555555555555555]","[32, 32]",0.2,0.0001,0,2,False,0.0,768
0,focal,0.529075,0.529075,"[0.5286259541984732, 0.5295238095238096]",0.238527,0.672598,0.642928,"[0.7458563535911602, 0.54]","[32, 32]",0.2,0.0001,1,2,False,0.0,768
0,focal,0.526215,0.525657,"[0.5419354838709677, 0.5093780848963475]",0.250377,0.661922,0.635239,"[0.7338935574229692, 0.5365853658536585]","[32, 32]",0.2,0.0001,12,2,False,0.0,768
0,focal,0.507150,0.506970,"[0.5163704396632367, 0.49757045675413025]",0.248288,0.686833,0.651914,"[0.7621621621621621, 0.5416666666666666]","[32, 32]",0.2,0.0001,123,2,False,0.0,768
0,focal,0.513823,0.513628,"[0.5038910505836576, 0.5233644859813084]",0.247281,0.676157,0.650598,"[0.7450980392156863, 0.5560975609756097]","[32, 32]",0.2,0.0001,1234,2,False,0.0,768


In [13]:
best_ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,input_dim
0,focal,0.515729,0.515359,"[0.5019607843137254, 0.5287569573283859]",0.246686,0.686833,0.656899,"[0.7582417582417582, 0.5555555555555555]","[32, 32]",0.2,0.0001,0,2,False,768
0,focal,0.529075,0.529075,"[0.5286259541984732, 0.5295238095238096]",0.238527,0.672598,0.642928,"[0.7458563535911602, 0.54]","[32, 32]",0.2,0.0001,1,2,False,768
0,focal,0.526215,0.525657,"[0.5419354838709677, 0.5093780848963475]",0.250377,0.661922,0.635239,"[0.7338935574229692, 0.5365853658536585]","[32, 32]",0.2,0.0001,12,2,False,768
0,focal,0.507150,0.506970,"[0.5163704396632367, 0.49757045675413025]",0.248288,0.686833,0.651914,"[0.7621621621621621, 0.5416666666666666]","[32, 32]",0.2,0.0001,123,2,False,768
0,focal,0.513823,0.513628,"[0.5038910505836576, 0.5233644859813084]",0.247281,0.676157,0.650598,"[0.7450980392156863, 0.5560975609756097]","[32, 32]",0.2,0.0001,1234,2,False,768


In [12]:
best_ffn_mean_history["f1"].mean()

0.5181377507114042

In [11]:
np.stack(best_ffn_mean_history["f1_scores"]).mean(axis=0)

array([0.51855674, 0.51771876])